<h1><center>Big Data Algorithms Techniques & Platforms</center></h1>

<h2>
<hr style=" border:none; height:3px;">
<center>Assignment 2: Introduction to Spark</center>
<hr style=" border:none; height:3px;">
</h2>

# 1. Introduction


<p align="justify">
<font size="3">
In this set of exercises you'll learn basic Spark programming skills that are necessary to develop simple, yet powerful, applications to be executed in a distributed environment.
</font>
</p>

<p align="justify">
<font size="3">
The assignment is presented in this __Jupyter Notebook__, an interface that offers support for text, code, images and other media. Essentially, a Jupyter Notebook consists of multiple _cells_, either containing some text, like the one that you are reading, or code that you can execute. 
</font>
</p>



In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar zxvf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local")
sc = SparkContext(conf = conf)
print("initialization successful!")

import numpy as np
import random as rn

seed_value=0
import os
os.environ['PYTHONHASHSEED']=str(seed_value)


# B. Data import


<p align="justify">
<font size="3">
Upload the folder data.zip inside the colab data folder and then execute the following code.
</font>
</p>



In [7]:
!apt-get install unzip
!unzip data.zip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 63 not upgraded.
Archive:  data.zip
replace data/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# C. Support functions 


<p align="justify">
<font size="3">
Some support functions are provided. Read carefully the signatures of the fuctions.
<ul>
<li> $remove\_non\_letters(word)$
<li> $load\_stopwords(stopwords\_file)$
<li> $preprocess(text, stopwords)$
<li> $word\_count(words)$
</ul>
</font>
</p>



In [9]:
import re
# Regular expression for removing all non-letter characters in the file.
regex = re.compile('[^a-zA-Z ]')


'''
Removes any non-letter character from the given word.

INPUT:
        word: A word

OUTPUT:
        the input word without the non-letter characters.

'''
def remove_non_letters(word):
    return regex.sub('', word)


'''
INPUT: 
        stopwords_file: name of the file containing the stopwords.
OUTPUT:
        a Python list with the stopwords read from the file.
'''
def load_stopwords(stopwords_file):
    stopwords = []
    with open(stopwords_file) as file:
        for sw in file:
            stopwords.append(sw.strip())
    return stopwords


'''
INPUT: 
        text: RDD where each element is a line of the input text file.
        stopwords: Python list containing the stopwords.
OUTPUT: 
        RDD where each element is a word from the input text file.
'''
def preprocess(text, stopwords) :
  words = text.flatMap(lambda line: line.split(" ")).map(lambda word: remove_non_letters(word)).filter(lambda word: len(word) > 0).map(lambda word: word.lower()).filter(lambda word: word not in stopwords)
  return words

'''
Returns how many times a word appears in a RDD 
INPUT:
        words: RDD, where each element is word from the input text file (preprocessing already done!).
OUTPUT:
        RDD, where each element is (w, occ), w is a word and occ the number of occurrences of w.
        The RDD is sorted by value in decreasing order.
'''

def word_count(words):    
    occs = words.map(lambda word: (word, 1))\
                .reduceByKey(lambda x, y: x+y)\
                .sortBy(lambda f: f[1], ascending=False)
    return occs

# Storing in stopwords the list of the stopwords that is provided
stopwords = load_stopwords("./data/stopwords.txt")





<hr style="border:solid 2px;">

##  Exercise 1

<p align="justify">
<font size="3">
In the folder _./data/bbc_ you'll find a collection of 50 documents from the BBC news website corresponding to stories in five topics. The five topics are:
<ul>
<li> _business_ 
<li>_entertainment_
<li> _politics_
<li> _sport_ 
<li> _tech_
</ul>

In the directory, the stories are text files (named: $\_001.txt\_$, $\_002.txt\_$, ...) organized into five directories, one for topic.
</font>
</p>

<p align="justify">
<font size="3">
In this exercise, we want to create an **inverted index**. An inverted index is an essential component of a search engine. In fact, given any word, the inverted index allows the search engine to quickly retrieve all documents containing that word.

An inverted index associates each word (you can find in the files) to the list of the names files the word occurs in.

More precisely, for each word, the inverted index will have a list of the names  of the files (path relative to the folder _./data_) that contain the word. 


(family, \[./data/bbc/tech/006.txt, ./data/bbc/entertainment/003.txt, ./data/bbc/entertainment/005.txt, ...\]
</font>
</p>

<p align="justify">
<font size="3">
The function $inverted\_index$ has the following input and output:
<ul>
    <li> **Input.** A RDD $files$, where each element is $(f, content)$, $f$ being the name of a text file in the collection and $content$ being the content of that file; 
a Python list $stopwords$, containing the most common English stopwords.
    <li> **Output.** A RDD, where each element is $(w, L)$, $w$ is a word and $L$ is the list of the names of the files containing $w$. The list must not contain duplicate file names.
</ul>
</font>
</p>

<p align="justify">
<font size="3" color='#91053d'>**Write the code of the function $inverted\_index()$. The function must apply a sequence of RDD transformations to:**
<ol>
  <li> split the content of each file into its constituent words.
  <li> lowercase each word.
  <li> remove the non-letter characters from each word (you can use the function $remove\_non\_letters$ defined in Exercise 1).
  <li> remove empty words.
  <li> remove the stopwords.
  <li> remove duplicate words.
</ol>
</font>
</p>
<hr style="border:solid 2px;">

In [10]:
'''
INPUT:
        files: RDD, each element is (f, content), where f is the name of a file in the collection and content is 
                its content.
        stopwords: a Python list containing the stopwords.

OUTPUT:y

        a RDD, each element is (w, L), where w is a word and L is the list of the names of the files containing
        w (without repetition).

'''

def inverted_index(files, stopwords):
    '''############## WRITE YOUR CODE HERE ##############'''

    #Split the content of each file into its constituent words and lowercase each word.
    
    # We first split the text into (f, w1), (f, w2) ... (f, wn) for all words in our file
    # Splitting into so many pairs modularizes our following tasks (since they only deal with words)
    # Likewise, splitting it into separate words allows for better paralellization in a distributed system
    # Especially if working with limited memory and extremelly large files
    line_split = files.flatMapValues(lambda text: text.split())
    
    # Applying lowercase transformation
    lower_case = line_split.mapValues(lambda w: w.lower())
    
    # We could also implement it as a one-liner
    #line_split = files.flatMapValues(lambda text: text.split()).mapValues(lambda w: w.lower())

    #remove the non-letter characters from each word (you can use the function  𝑟𝑒𝑚𝑜𝑣𝑒_𝑛𝑜𝑛_𝑙𝑒𝑡𝑡𝑒𝑟𝑠  defined in Exercise 1).
    no_non_letters = lower_case.mapValues(remove_non_letters)
    
    #remove empty words
    no_empty = no_non_letters.filter(lambda pair: pair[1]) 
    
    #remove the stopwords
    no_stop_words = no_empty.filter(lambda pair: pair[1] not in stopwords)

    #remove duplicate words
    duplicates_removed = no_stop_words.distinct()

    #produce the inverted index dictionary
    output = duplicates_removed.map(lambda pair: (pair[1], pair[0])).groupByKey().mapValues(list)

    return output

    '''############## END OF THE EXERCISE ##############'''

'''
INPUT:
        iindex: RDD containing the inverted index, as returned by the function inverted_index.
        word: a word.

OUTPUT:
        prints the list of the files contain the given word.
'''
def lookup(iindex, word):
    ld = iindex.sortByKey().lookup(word)
    if len(ld) > 0:
        print("The following documents contain the word '",word,"'")
        for d in sorted(ld[0]):
            print(os.path.relpath(d[5:], os.getcwd()))
    else:
        print("No documents contain the word '",word,"'")

####################   GOOD TO KNOW  ####################
# The Spark function wholeTextFiles loads into a RDD the content of the text files contained
# in the given directory.
# Each item of the RDD is a pair (f, content), where f is the name of a file and content is the content
# of the file.
#######################################################
file_collection = sc.wholeTextFiles("./data/bbc/*")        
iindex = inverted_index(file_collection, stopwords)
lookup(iindex, "family")

################# EXPECTED OUTPUT #################
#
# data/bbc/entertainment/002.txt
# data/bbc/entertainment/003.txt
# data/bbc/entertainment/005.txt
# data/bbc/politics/001.txt
# data/bbc/sport/004.txt
# data/bbc/tech/004.txt
# data/bbc/tech/006.txt
#
###################################################


The following documents contain the word ' family '
data/bbc/entertainment/002.txt
data/bbc/entertainment/003.txt
data/bbc/entertainment/005.txt
data/bbc/politics/001.txt
data/bbc/sport/004.txt
data/bbc/tech/004.txt
data/bbc/tech/006.txt


<hr style="border:solid 2px;">

##  Exercise 2

<p align="justify">
<font size="3">
Given the BBC collection, we want to calculate the **co-occurrence matrix** $M$, such that $M[w_1][w_2]$ is the number of documents in which two words $w_1$ and $w_2$ appear in the same document (it does not matter if they are consecutive or not).
</font>
</p>

<p align="justify">
<font size="3">
The function $co\_occurrence\_matrix()$ has the following input and output:
<ul>
 <li> **Input.** A RDD $files$ and a Python list $stopwords$, as in the previous exercise.
 <li> **Output.** A RDD, where each element is $((w_1, w_2), occ)$, where $w_1$ and $w_2$ are words and $occ$ is the number of files in which the two words appear together.
</ul>
As in the case of the function $inverted\_index()$, words must be lowercases, non-letter characters, empty words and stopwords should be removed.
</font>
</p>

<p align="justify">
<font size="3" color='#91053d'>**Write the code of the function $co\_occurrence\_matrix()$. You can draw inspiration from the MapReduce algorithms that we discussed in class. Also, you can use the already implemented function $create\_pairs()$ to generate all the possible pairs from a list of words. The function assumes that the words in the input list are sorted lexicographically.**
<br>
</font>
</p>

<hr style="border:solid 2px;">

In [26]:
'''
INPUT:
        words: Python list containing words. IMPORTANT: the function assumes that the 
        list is sorted in lexicographic order.
OUTPUT:
        Python list containing all possible pairs from the given list.
'''
def create_pairs(words):
    n = len(words)
    output = []
    for i in range(0, n):
        for j in range(i+1, n):
            output.append((words[i], words[j]))
    return output

'''
INPUT:
        files: RDD, each item is (f, content), where f is the name of a file and line is the content of the file.
        stopwords: A RDD, each item is ((w1, w2), occ), where w1 and w2 are words and occ is the number of
                    files in which w1 and w2 appear together.
'''
def co_occurrence_matrix(files, stopwords):
    '''############## WRITE YOUR CODE HERE ##############'''
    # For this exercise, since it does not specify specific variable and describes continuous transformations on the same RDD
    # We will encode all our spark code into successive transformations of the same RDD to better display that everything is processed in-place
    output = (files.flatMapValues(lambda text: text.split()) # Splitting into (L, w1), (L, w2)... 
                # As discussed before, possibly necessary for very large files and limited memory, also provides better paralelization
               .mapValues(lambda w: w.lower()) # Lowercasing the words
               .mapValues(remove_non_letters) # Removing non word characters
               .filter(lambda pair: pair[1]) # Removing empty words -> pair[1] empty returns false -> filtered
               .filter(lambda pair: pair[1] not in stopwords) # Removing stop words
               .distinct() # Removing duplicates
               
               .groupByKey().mapValues(list) # Regrouping our filtered files per link as a list 
               # (L, [w1, w2, ..., wN])
               
               .mapValues(sorted) # Sorting the list of words alphabetically (lexicographic order)
               
               .flatMapValues(create_pairs) # Using the create_pairs function in order all (L, (w1, w2))
               # for any w1 < w2 and w1 and w2 belonging to the same document L
               # Once again, flatmapping might be use in very large files and for paralellization

               .map(lambda pair_rdd: (pair_rdd[1], 1)) # Transforming (L, (w1, w2)) into ((w1,w2), 1)
               # Will allow for transitioning into the classic word-counting example of MapReduce, but with a tuple instead of a single word

               # Using an addition reduce to count the quantity of the pairs
               .reduceByKey(lambda x, y: x + y))
    
    return output
    '''############## END OF THE EXERCISE ##############'''


file_collection = sc.wholeTextFiles("./data/bbc/*")
output = co_occurrence_matrix(file_collection, stopwords)    
output.takeOrdered(10, key = lambda x: -x[1])

################# EXAMPLE OF FORMAT FOR THE EXPECTED OUTPUT #################
################# THIS IS NOT THE SOLUTION #################
#
#[(('a', 'b'), 3),
# (('c', 'f'), 12),
# ... ]
#
###################################################


[(('also', 'said'), 24),
 (('said', 'world'), 20),
 (('new', 'said'), 19),
 (('said', 'year'), 17),
 (('also', 'world'), 17),
 (('one', 'said'), 16),
 (('last', 'said'), 15),
 (('said', 'set'), 15),
 (('said', 'time'), 15),
 (('back', 'said'), 14)]

<hr style=" border:solid 2px;">

##  Exercise 3 - OPTIONAL - enjoy with what you just wrote

<p align="justify">
<font size="3">
We want to code a function $term\_freq$ that computes the frequency of each word in a 
text document. 
More precisely, given a document $d$ and a word $w$ in that document, we want to 
compute its frequency $tf(w, d)$, as follows:
    
<p>    
$$ tf(w, d) = \frac{f_{w, d}}{\sum\limits_{w^\prime \in d} f_{w^\prime, d}}$$
</p>

where $f_{w, d}$ is the number of occurrences of word $w$ in $d$.
</font>
</p>

<p>
<font size="3">
The function $term\_freq$ has the following input and output:
<ul>
<li> **Input.** A RDD $words$, where each element is a word in a text document $d$ (pre-processing already done).
<li> **Output.** A RDD, where each element is a key-value pair $(w, tf(w, d))$.
</ul>
</font>
</p>
<p align="justify">
<font size="3" color='#91053d'>**Write the code of the function $term\_freq$. You can take advantage of the 
    function $word\_count$.**
</font>
</p>

<hr style=" border:solid 2px;">

In [50]:
def term_freq(words):
    '''############## WRITE YOUR CODE HERE ##############'''
    counts = word_count(words)
    
    sum_all = counts.map(lambda x: x[1]).reduce(lambda x, y: x+y)

    raw_counts = counts.map(lambda x: (x[0], x[1] / sum_all)) 
    
    return raw_counts

    '''############## END OF THE EXERCISE ##############'''
 
text = sc.textFile('./data/bbc/politics/001.txt')
words = preprocess(text, stopwords)
tf = term_freq(words)
tf.take(5)

################# EXPECTED OUTPUT #################
################# THIS IS NOT THE SOLUTION #################
#
#[('a', 0,333),
# ('b', 0.032),
# ... ]
#
###################################################

[('pay', 0.045081967213114756),
 ('maternity', 0.036885245901639344),
 ('months', 0.036885245901639344),
 ('said', 0.036885245901639344),
 ('plans', 0.020491803278688523)]